<a href="https://colab.research.google.com/github/KareemYasser20/Pneumoray/blob/main/ResNet50_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import numpy as np 
import pandas as pd 
from os import listdir, makedirs
from os.path import join, exists, expanduser
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
from keras.utils.data_utils import Sequence
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import confusion_matrix
import itertools
from tensorflow.keras.applications import ResNet50

In [ ]:
img_width, img_height = 224, 224
train_data = '/content/drive/MyDrive/MixData/train'
test_data = '/content/drive/MyDrive/MixData/test'
val_data = '/content/drive/MyDrive/MixData/val'

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

train_datagen = ImageDataGenerator(
      rescale=1./255,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
          )
test_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.5,
        zoom_range=0.1,
        # rotation_range=0.9,
        fill_mode='nearest',
        # height_shift_range=0.2,
        width_shift_range=0.2,
        horizontal_flip=True,
)


val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(train_data, batch_size = 16, class_mode = 'categorical', target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory(val_data, batch_size = 4, class_mode = 'categorical', target_size = (224, 224))

test_generator = test_datagen.flow_from_directory(test_data, batch_size = 8, class_mode = 'categorical', target_size = (224, 224))


In [ ]:
base_model = ResNet50( include_top=False, weights= "imagenet")

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

dropout = 0.1

model_resnet = Sequential()
model_resnet.add(Conv2D(64, (3, 3),input_shape=(224, 224,3)))
model_resnet.add(Activation('relu'))
model_resnet.add(MaxPooling2D(pool_size=(2, 2)))
model_resnet.add(Dropout(dropout))

model_resnet.add(Conv2D(128, (3, 3)))
model_resnet.add(Activation('relu'))
model_resnet.add(MaxPooling2D(pool_size=(2, 2)))
model_resnet.add(Dropout(dropout))

model_resnet.add(Conv2D(256, (3, 3)))
model_resnet.add(Activation('relu'))
model_resnet.add(MaxPooling2D(pool_size=(2, 2)))
model_resnet.add(Dropout(dropout))


model_resnet.add(Flatten())  
model_resnet.add(Dense(512))
model_resnet.add(Activation('relu'))
model_resnet.add(Dropout(dropout))
model_resnet.add(Dense(3))
model_resnet.add(Activation('softmax'))


In [ ]:
optimizer1 = 'adam'
model_resnet.compile(optimizer=optimizer1,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
resnet_history = model_resnet.fit(train_generator, validation_data = validation_generator, epochs = 16 , shuffle = True, verbose = 1 ) 

In [ ]:
base_model.summary()

In [ ]:
data_evaluate = model_resnet.evaluate(test_generator)

In [ ]:
pd.DataFrame(resnet_history.history).plot()
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(resnet_history.history['loss'], label='Training Loss')
plt.plot(resnet_history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(resnet_history.history['accuracy'], label='Training Accuracy')
plt.plot(resnet_history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# # show the confusion matrix of our predictions

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# compute confusion matrix
cnf_matrix = confusion_matrix(y_true, y_pred)
np.set_printoptions(precision=2)

# plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()

In [ ]:
#Checking the calss indices: {'BAC_PNEUMONIA': 0, 'NORMAL': 1, 'VIR_PNEUMONIA': 2}
proba_predictions= model_resnet.predict(test_generator)
# print(proba_predictions)
dict_classes= train_generator.class_indices
dict_classes

In [ ]:
y_true= test_generator.classes
# y_predictions= (proba_predictions >0.6).astype('int32')
print(classification_report(y_true,np.argmax(proba_predictions, axis=1) ))